In [48]:
import io, sys, math, re
from collections import defaultdict
import numpy as np

In [49]:
# dataloader

def load_data(filename):
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    vocab = defaultdict(lambda:0)
    for line in fin:
        sentence = line.split()
        data.append(sentence)
        for word in sentence:
            vocab[word] += 1
    return data, vocab

In [50]:
def remove_rare_words(data, vocab, mincount):
    ## FILL CODE
    # replace words in data that are not in the vocab 
    # or have a count that is below mincount
    data_with_unk = data[:]
    for w1, sentence  in enumerate(data):
        for w2, word in enumerate(sentence):
            if vocab[word] < mincount or word not in vocab :
                data_with_unk[w1][w2] = '<unk>'
            
    
    return data_with_unk


In [51]:
# LOAD DATA
print("load training set")
train_data, vocab = load_data("train2.txt")
## FILL CODE 
# If you have a Out of Vocabulary error (OOV) 
# call the function "remove_rare_words" to replace 
# rare words with <unk> in the dataset
train_data = remove_rare_words(train_data, vocab, mincount=15)

print("load validation set")
valid_data, _ = load_data("valid2.txt")
## FILL CODE 
# If you have a Out of Vocabulary error (OOV) 
# call the function "remove_rare_words" to replace 
# OOV with <unk> in the dataset
valid_data = remove_rare_words(valid_data, vocab, mincount=15)

load training set
load validation set


In [34]:
# for sentence in train_data:
#         sentence = tuple(sentence)
#         total_number_words +=len(sentence)

# #         for i, words in enumerate(sentence):
            

#         print(total_number_words)

In [35]:
import ipdb

In [52]:
def build_ngram(data, n):
    total_number_words = 0
    counts = defaultdict(lambda: defaultdict(lambda: 0.0))

    for sentence in data:
        sentence = tuple(sentence)
        ## FILL CODE
        # dict can be indexed by tuples
        # store in the same dict all the ngrams
        # by using the context as a key and the word as a value
        for j in range(n):
            for i in range(len(sentence)-j):
                total_number_words +=1
                counts[sentence[i:(i+j)]][sentence[i+j]] +=1
    total_number_words = total_number_words / n
            
#             ipdb.set_trace()
        

    prob  = defaultdict(lambda: defaultdict(lambda: 0.0))
    ## FILL CODE
    # Build the probabilities from the counts
    # Be careful with how you normalize!
    
    for  word  in counts:
        for w in counts[word]:
            prob[word][w] = counts[word][w]/sum(counts[word].values())

    return prob

In [57]:
# RUN TO BUILD NGRAM MODEL

n = 2
print("build ngram model with n = ", n)
model = build_ngram(train_data, n)
model;

build ngram model with n =  2


In [60]:
def get_prob(model, context, w):
    ## FILL CODE
    # code a recursive function over 
    # smaller and smaller context
    # to compute the backoff model
    # Bonus: You can also code an interpolation model this way
    if model[context][w] !=0:
        return model[context][w]
    else :
        return 0.4*get_prob(model, context[1:],w)
        
        

def perplexity(model, data, n):
    ## FILL CODE
    # Same as bigram.py
    perp = 0
    for sentence in data:
        sentence = tuple(sentence)
        prob = 0
        for i in range(1, len(sentence)):
            if i>= n-1:
                prob += np.log(get_prob(model, sentence[i-n+1:i], sentence[i]))
            else:
                prob += np.log(get_prob(model, sentence[:i], sentence[i]))
        prob = -prob/len(sentence)
        perp += prob
    
    return np.exp(perp/len(data))

In [61]:
# COMPUTE PERPLEXITY ON VALIDATION SET

print("The perplexity is", perplexity(model, valid_data, n))

The perplexity is 32.21820986937871


In [56]:
def get_proba_distrib(model, context):
    ## FILL CODE
    # code a recursive function over context
    # to find the longest available ngram 
    if context  in model:
        return context
    else:
        return get_proba_distrib(model, context[1:])

def generate(model):
    ## FILL CODE
    # generate a sentence. A sentence starts with a <s> and ends with a </s>
    # Possiblly a use function is:
    #   np.random.choice(x, 1, p = y)
    # where x is a list of things to sample from
    # and y is a list of probability (of the same length as x)
    sentence = ['<s>']
    for length_sentence in range(20):
        if sentence[-1]=='</s>':
            break
        x= list(model[get_proba_distrib(model, tuple(sentence))].keys())
        y= list(model[get_proba_distrib(model, tuple(sentence))].values())
        word = np.random.choice(x, 1, p=y)
        sentence.append(word[0])
    if sentence[-1] !='</s>':
        sentence.append('</s>')
    return sentence

In [47]:
# GENERATE A SENTENCE FROM THE MODEL

print("Generated sentence: ",generate(model))

Generated sentence:  ['<s>', 'he', '<unk>', 'his', '<unk>', '<unk>', 'in', '<unk>', 'the', 'same', '<unk>', '</s>']
